In [120]:
from typing import List, Tuple, Any
from datetime import datetime
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import gym
from gym import wrappers
import matplotlib.pyplot as plt
from pprint import pprint
from datetime import datetime, timedelta
import operate


In [2]:
def get_unix(date: str, time: str) -> int:
    format_date = "%d/%m/%Y"
    format_time = "%H:%M:%S.%f000"
    date_obj = datetime.strptime(date, format_date)
    time_obj = datetime.strptime(time, format_time)
    almost = datetime.combine(date_obj.date(), time_obj.time())
    return int(almost.timestamp())
class Tick:
    def __init__(self, line: str) -> None:
        splitted_line = line.split(',')
        self.date = splitted_line[0]
        self.time = splitted_line[1]
        self.bid = float(splitted_line[2].strip())
        self.ask = float(splitted_line[4].strip())
        self.unix_code = get_unix(splitted_line[0], splitted_line[1])
class Strategy:
    def __init__(self) -> None:
        self.num_strats = 4
        pass 

    def Von_neuman( self, id: int, prices: List[ float ], tres_hold: float, acumulated_side_bool: bool, cant_part = 8 ) -> int:
        prices = prices[::-1]
        if ( id == 0 ) or ( acumulated_side_bool is False ) :
            up_down = [ ]
            if cant_part < len(prices) :
                for index_groups in range(cant_part - 1) :
                    try:
                        if prices[index_groups * 100] < prices[(index_groups + 1) * 100] :
                            up_down.append(1)
                        elif prices[index_groups * 100] > prices[(index_groups + 1) * 100] :
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try :
                    avg = sum(up_down) / len(up_down)
                    if avg > tres_hold:
                        return -1
                    else:
                        return 0
                except ZeroDivisionError :
                    return 0
            else:
                return 0
        elif ( id > 0 ) and ( acumulated_side_bool is True ) :
            up_down = []
            if cant_part < len(prices):
                for index_groups in range(cant_part - 1) :
                    try:
                        if prices[index_groups * 10] < prices[(index_groups + 1) * 10] :
                            up_down.append(1)
                        elif prices[index_groups * 10] > prices[(index_groups + 1) * 10] :
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try :
                    avg = sum( up_down ) / len( up_down )
                    if -tres_hold <= avg <= tres_hold :
                        return 0 
                    elif avg > 0 :
                        return -1 
                    else :
                        return 1 
                except ZeroDivisionError:
                    return 0
            else:
                return 0

                
def compute_balance(orders_list: List[Any], tick: Tick) -> float:
    open_balance = 0
    for line in orders_list:
        if len(line) > 2:
            side = line[ 0 ]
            number_units = line[1]
            bid_price, ask_price = line[2], line[3]
            last_bid, last_ask = tick.bid, tick.ask
            profit_loss = 0
            if side == -1:
                profit_loss = number_units * last_bid - number_units * ask_price
            elif side == 1:
                profit_loss = number_units * bid_price - number_units * last_ask
            open_balance += profit_loss
    return open_balance * 1
def compute_drawdown(path_to_use: str, orders_list: List[Any], tick: Tick) -> float:
    if len(orders_list) > 1 :
        last_balance = compute_balance(orders_list, tick)
        balances = []
        with open(path_to_use, 'r') as file_reader:
            for line in file_reader:
                balances.append(float(line.split(':')[6]))
        balances.append(last_balance)
        max_drawdown = 0
        drawdown = 0
        peak = balances[0]
        for balance in balances:
            if balance > peak:
                peak = balance
            drawdown = peak - balance
            if drawdown > max_drawdown:
                max_drawdown = drawdown
        return max_drawdown
    else:
        return 0
def Register(path_to_use: str, line: List[Any]) -> None:
    with open(path_to_use, 'a') as file_in_process:
        for value in line:
            file_in_process.writelines(f'{value}:')
        file_in_process.writelines('\n')
 
def file_content(path_to_use: str) -> str:
    if os.path.exists(path_to_use):
        with open(path_to_use, 'r') as file_in_process:
            return file_in_process.read()
    else:
        return ""
def time_signal_detector( current_tick: Tick, previous_tick: Tick, pivot_unix: int ) -> bool:
    current_unix = current_tick.unix_code
    previous_unix = previous_tick.unix_code
    if current_unix <= pivot_unix:
        return True
    elif previous_unix <= pivot_unix < current_unix:
        return False
def State_detector(acumulated_side) -> bool:
    if acumulated_side >= 0 :
          return False
    else :
          return True
time_frames = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 
               98, 105, 112, 119, 126, 133, 140, 147, 154, 161, 168, 
               175, 182, 189, 196, 203, 210, 217, 224, 231, 238, 245,
               252, 259, 266, 273, 280, 287, 294, 301, 308, 315, 322,
               329, 336, 343, 350, 357, 364, 371, 378, 385, 392, 399,
               406, 413, 420, 427, 434, 441, 448, 455, 462, 469, 476,
               483, 490, 497, 504, 511, 518, 525, 532, 539, 546, 553,
               560, 567, 574, 581, 588, 595, 602, 609, 616, 623, 630,
               637, 644, 651, 658, 665, 672, 679, 686, 693]
dict_for_tres_hold = {  0.07: 'A',
                        0.08: 'B',
                        0.09: 'C',
                        0.1: 'D',
                        0.15: 'E',
                        0.16: 'F',
                        0.17: 'G',
                        0.18: 'H',
                        0.19: 'I',
                        0.2: 'J',
                        0.25: 'K' }
dict_for_decision =  {  1: 'sell',
                       -1: 'buy'  }
dict_for_time_frames = {}
for i,x in enumerate(time_frames):
    dict_for_time_frames[f'{x}'] = f'Tf_{i}'

def Process_day(time_frame: str, path_of_current_day: str, day: str, folder_for_results : str ,Strat: Strategy, sum_side_by_day = 0) -> str:
    list_for_balance = []
    prices = []
    saving_path ='Vn_'+ dict_for_time_frames[time_frame]+'_'+day
    
    with open(path_of_current_day, 'r') as current_day:
        current_day_list = current_day.readlines()
        last_hour = int(current_day_list[-2].split(',')[1].split(':')[0]) + 1
        current_hour = 0
        int_part, *decimal_part = map(int, time_frame.split('.'))
        increment_minutes = int_part
        increment_seconds = decimal_part[0] * 6 if decimal_part else 0

        current_minute = 0
        current_seconds = 0
        pivot_time = f'{current_hour:02}:{current_minute:02}:{current_seconds:02}.000000000'
        id = 0

        for index in range(1, len(current_day_list)):
            current_tick = Tick(current_day_list[index])
            previous_tick = Tick(current_day_list[index - 1])
            pivot_unix = get_unix(current_tick.date, pivot_time)

            if time_signal_detector(current_tick, previous_tick, pivot_unix):
                prices.append(np.random.choice([previous_tick.bid, current_tick.ask]))
            else:
                selected_tres_hold = np.random.choice([0.07, 0.08, 0.09, 0.1, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.25])
                side = Strat.Von_neuman(id, prices, selected_tres_hold, State_detector(sum_side_by_day))
                sum_side_by_day += side

                if side != 0:
                    list_for_balance.append([side, 1, previous_tick.bid, previous_tick.ask])
                    P_L = compute_balance(list_for_balance, previous_tick)
                    drawdown = compute_drawdown(saving_path, list_for_balance, previous_tick)
                    Register(saving_path, [id, dict_for_tres_hold[selected_tres_hold], dict_for_decision[side], 1, previous_tick.bid, previous_tick.ask, P_L , drawdown ])
                    id += 1

                
                current_seconds += increment_seconds
                if current_seconds >= 60:
                    current_seconds -= 60
                    current_minute += 1

                current_minute += increment_minutes
                if current_minute >= 60:
                    current_hour += current_minute // 60
                    current_minute %= 60

                if current_hour >= last_hour:
                    print( os.path.join(folder_for_results,saving_path))
                    with open( os.path.join(folder_for_results,saving_path),'w') as exit_file :
                        exit_file.write(file_content(saving_path))
                    if os.path.exists(saving_path) :
                        os.remove(saving_path)
                    return

                pivot_time = f'{current_hour:02}:{current_minute:02}:{int(current_seconds):02}.000000000'
                prices = []
        #return 


In [3]:
folder = "E:\\B-Module\\TXTs"
folder_for_results = [ "E:\\B-Module\\results\\Von_Neuman" ]
archives_in_folder = os.listdir( folder )
for archive_name in archives_in_folder:
    if archive_name.endswith('.txt'):
        complete_route = os.path.join(folder, archive_name)
        for time_frame in time_frames:
            print(f'{time_frame} : {complete_route}')
            Strat = Strategy( )
            Process_day(str(time_frame), complete_route, archive_name, folder_for_results[0], Strat)
            '''
            for path_of_analized_day, folder_for_results in zip(group_path_of_analized_day, group_folder_for_results ) :
                place_path = os.path.join(folder_for_results, path_of_analized_day)
                with open(place_path, 'w') as archive:
                        archive.write(file_content(path_of_analized_day))
                print(f'The following path has been saved : {place_path}')
                if os.path.exists(path_of_analized_day):
                    os.remove(path_of_analized_day)
                else:
                    pass
            '''

7 : E:\B-Module\TXTs\01-02-2023.txt


E:\B-Module\results\Von_Neuman\Vn_Tf_0_01-02-2023.txt
14 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_1_01-02-2023.txt
21 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_2_01-02-2023.txt
28 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_3_01-02-2023.txt
35 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_4_01-02-2023.txt
42 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_5_01-02-2023.txt
49 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_6_01-02-2023.txt
56 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_7_01-02-2023.txt
63 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_8_01-02-2023.txt
70 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_9_01-02-2023.txt
77 : E:\B-Module\TXTs\01-02-2023.txt
E:\B-Module\results\Von_Neuman\Vn_Tf_10_01-02-2023.txt
84 : E:\B-Module\TXTs\01-02-2023.tx

KeyboardInterrupt: 

In [69]:
results_Von_Neuman = "E:\\B-Module\\results\\Von_Neuman"
archives_in_folder = os.listdir( results_Von_Neuman )
print(len(archives_in_folder))
for archive_name in archives_in_folder :
    if archive_name.endswith('.txt') :
        complete_route = os.path.join(results_Von_Neuman, archive_name)



650


## Analyzer

In [113]:
def compute_balance_for_big_file(orders_list: List[Any], last_bid : float,  last_ask : float ) -> float :
    open_balance = 0
    for line in orders_list:
        if len(line) > 2:
            side = line[0]
            number_units = int(line[1])
            bid_price, ask_price = float(line[2]), float(line[3])
            profit_loss = 0
            last_bid = float( last_bid )
            last_ask = float(last_ask)
            if side == 'buy' :
                profit_loss = number_units * (last_bid) - number_units * ask_price
            elif side == 'sell' :
                profit_loss = number_units * bid_price - number_units * last_ask
            open_balance += profit_loss
    return open_balance * 12500

def compute_drawdown_for_big_file( balances : List[float] ) -> float:
    if len( balances ) > 1 :
                max_drawdown = 0
                drawdown = 0
                peak = balances[0]
                for balance in balances :
                    if balance > peak :
                        peak = balance
                    drawdown = peak - balance
                    if drawdown > max_drawdown:
                        max_drawdown = drawdown
                return max_drawdown
    else:
        return 0
def Register(path_to_use: str, line: List[Any]) -> None:
    with open(path_to_use, 'a') as file_in_process:
        for value in line:
            file_in_process.writelines(f'{value}:')
        file_in_process.writelines('\n')
def file_content(path_to_use: str) -> str:
    if os.path.exists(path_to_use):
        with open(path_to_use, 'r') as file_in_process:
            return file_in_process.read()
    else:
        return ""
def chronological_organizer(*, module_id, folder) :
    if module_id == 'Vn' :
        archives_in_folder = os.listdir( folder )####
        df = { 'Archive':[ ], 'Month':[ ],'Year':[],'Day':[] }
        for archive in archives_in_folder :
            if archive.endswith('.txt') :
                rout = os.path.join( folder, archive )
                with open( rout, 'r' ) as reading_analysis :
                    lines = reading_analysis.readlines( )
                    if len(lines) > 0:
                            df['Archive'].append(archive)
                            if len(archive) == 22 :
                                df['Day'].append(archive[8:10])
                                df['Month'].append(archive[11:13])
                                df['Year'].append(archive[14:18])
                            else :
                                df['Day'].append(archive[9:11])
                                df['Month'].append(archive[12:14])
                                df['Year'].append(archive[15:19])
        df= pd.DataFrame(df)
        df= df.sort_values( by= ['Year','Month','Day'])
        return df.Archive

def merge_txt_files(*,chronological_order, Tf, folder) :
    content = ""   
    list_for_balance = [ ]
    list_for_drawdown = [ ]
    for archive_name in chronological_order :
        print((archive_name))
        if len(archive_name) == 22:
            print(22)
            if archive_name.endswith('.txt') and Tf == (archive_name)[6:7]:
                complete_route = os.path.join( folder, archive_name )
                with open( complete_route, 'r' ) as current_day :
                    lines = current_day.readlines( )
                    if len( lines ) > 0 :
                        for order in lines :
                            splitted = order.split( ':' )
                            list_for_balance.append( splitted[ 2: 6 ] )
                            P_L = compute_balance_for_big_file(list_for_balance, splitted[4], splitted[5] )
                            list_for_drawdown.append(P_L)
                            drawdown = compute_drawdown_for_big_file( list_for_drawdown )
                            content +=f'{splitted[0]}:{splitted[1]}:{splitted[2]}:{splitted[3]}:{splitted[4]}:{splitted[5]}:{P_L}:{drawdown}\n'
        if len(archive_name) == 23:
            if archive_name.endswith('.txt') and Tf == (archive_name)[6:8]:
                complete_route = os.path.join( folder, archive_name )
                with open( complete_route, 'r' ) as current_day :
                    lines = current_day.readlines( )
                    if len( lines ) > 0 :
                        for order in lines :
                            splitted = order.split( ':' )
                            list_for_balance.append( splitted[ 2: 6 ] )
                            P_L = compute_balance_for_big_file(list_for_balance, splitted[4], splitted[5] )
                            list_for_drawdown.append(P_L)
                            drawdown = compute_drawdown_for_big_file( list_for_drawdown )
                            content +=f'{splitted[0]}:{splitted[1]}:{splitted[2]}:{splitted[3]}:{splitted[4]}:{splitted[5]}:{P_L}:{drawdown}\n'
    return content

In [118]:
#folder_for_summaries = input("Write the path where u want to place the summary by profile : ")
order = chronological_organizer( module_id = 'Vn', folder = results_Von_Neuman )
for i in range(99):
    path_of_analized_day = f'Vn_Tf_{i}_sum.txt'
    content = merge_txt_files(chronological_order = order, Tf = str(i), folder = results_Von_Neuman )
    with open(os.path.join(results_Von_Neuman,path_of_analized_day),'w') as writing_archive :
        writing_archive.write(content)



Vn_Tf_0_01-05-2022.txt
22
Vn_Tf_10_01-05-2022.txt
Vn_Tf_11_01-05-2022.txt
Vn_Tf_14_01-05-2022.txt
Vn_Tf_15_01-05-2022.txt
Vn_Tf_17_01-05-2022.txt
Vn_Tf_18_01-05-2022.txt
Vn_Tf_19_01-05-2022.txt
Vn_Tf_1_01-05-2022.txt
22
Vn_Tf_20_01-05-2022.txt
Vn_Tf_21_01-05-2022.txt
Vn_Tf_23_01-05-2022.txt
Vn_Tf_24_01-05-2022.txt
Vn_Tf_26_01-05-2022.txt
Vn_Tf_27_01-05-2022.txt
Vn_Tf_29_01-05-2022.txt
Vn_Tf_2_01-05-2022.txt
22
Vn_Tf_34_01-05-2022.txt
Vn_Tf_36_01-05-2022.txt
Vn_Tf_39_01-05-2022.txt
Vn_Tf_3_01-05-2022.txt
22
Vn_Tf_40_01-05-2022.txt
Vn_Tf_41_01-05-2022.txt
Vn_Tf_42_01-05-2022.txt
Vn_Tf_43_01-05-2022.txt
Vn_Tf_49_01-05-2022.txt
Vn_Tf_4_01-05-2022.txt
22
Vn_Tf_50_01-05-2022.txt
Vn_Tf_53_01-05-2022.txt
Vn_Tf_55_01-05-2022.txt
Vn_Tf_56_01-05-2022.txt
Vn_Tf_5_01-05-2022.txt
22
Vn_Tf_6_01-05-2022.txt
22
Vn_Tf_73_01-05-2022.txt
Vn_Tf_74_01-05-2022.txt
Vn_Tf_76_01-05-2022.txt
Vn_Tf_78_01-05-2022.txt
Vn_Tf_7_01-05-2022.txt
22
Vn_Tf_80_01-05-2022.txt
Vn_Tf_85_01-05-2022.txt
Vn_Tf_86_01-05-2022.txt


In [117]:
for i in range(99):
    path_of_analized_day = f'Vn_Tf_{i}_sum.txt'
    if os.path.exists(os.path.join(results_Von_Neuman,path_of_analized_day)) :
        os.remove(os.path.join(results_Von_Neuman,path_of_analized_day))

In [124]:
('Vn_Tf_0_sum.txt')[6:7]

'0'

In [133]:
df = {'Profile':[],'Drawdown':[],'Balance':[]}

for index in range(99):
  archive = f'Vn_Tf_{index}_sum.txt'  
  if archive.endswith('.txt') :
    complete_route = os.path.join(results_Von_Neuman,archive)
    with open(complete_route, 'r') as reading :
      all_list = reading.readlines()
      if len(all_list) > 0 :
        last_line_splitted = all_list[-1].split(':')
        if len(archive)==15 :
          profile = archive[6:7]
        else :
          profile = archive[6:8]
        df['Profile'].append((profile))
        df['Balance'].append(float(last_line_splitted[6]))
        df['Drawdown'].append(float(last_line_splitted[7]))
      else :
        print(all_list)
df = pd.DataFrame(df)
df = df.sort_values(by="Profile")
df.head(85)

[]
[]
[]


,Profile,Drawdown,Balance
0,0,11296.25,-1508.75
1,1,15048.75,8655.00
10,10,3030.00,-316.25
11,11,7083.75,2851.25
12,12,2341.25,-1531.25
...,...,...,...
81,83,0.00,892.50
82,84,0.00,915.00
83,85,756.25,-757.50
84,86,0.00,611.25


In [135]:
# Crear la gráfica de dispersión
fig = px.scatter(df, 
                 x="Balance", y="Drawdown", 
                 color="Profile", 
                 hover_name="Profile", 
                 template='plotly_white')

# Actualizar la escala de colores
fig.update_traces(marker=dict(colorscale=green_scale))

# Añadir el título
fig.update_layout(title='Drawdown x Balance x Profile')

# Añadir punto destacado
highlighted_point = {
    "x": [sum(df['Balance']) / len(df['Balance'])],
    "y": [sum(df['Drawdown']) / len(df['Drawdown'])],
    "text": ["Punto Destacado"],
    "marker": {"color": "red", "size": 20, "symbol": "star"}
}

fig.add_trace(
    go.Scatter(
        x=highlighted_point["x"],
        y=highlighted_point["y"],
        mode='markers+text',
        text=highlighted_point["text"],
        textposition='top center',
        marker=highlighted_point["marker"],
        name='Punto Destacado'
    )
)

# Calcular el centroide
X = sum(df['Balance']) / len(df['Balance'])
Y = sum(df['Drawdown']) / len(df['Drawdown'])
print(f'centroide X: {X}, Y: {Y}')

# Mostrar la gráfica
fig.show()


centroide X: 489.4921874999982, Y: 1759.7005208333333


In [136]:
489.49218749999824/1759.7005208333333

0.2781678937437557